In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
import transformers

# set to only report critical errors to avoid excessing logging
transformers.utils.logging.set_verbosity(50)

In [4]:
from nlpsig_networks.scripts.fine_tune_bert_classification import (
    fine_tune_transformer_average_seed,
)

In [5]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [6]:
%run load_sbert-embeddings.py

In [7]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


## Baseline: Fine-tine BERT for classification

In [8]:
num_epochs = 5
seeds = [1, 12, 123]
validation_metric = "f1"

In [12]:
kwargs = {
    "num_epochs": num_epochs,
    "pretrained_model_name": "bert-base-uncased",
    "df": df_rumours,
    "feature_name": "text",
    "label_column": "label",
    "seeds": seeds,
    "device": device,
    "batch_size": 8,
    "split_ids": split_ids,
    "k_fold": True,
    "validation_metric": validation_metric,
    "verbose": False,
}

## Focal Loss

In [13]:
loss = "focal"
gamma = 2

In [14]:
bert_classifier = fine_tune_transformer_average_seed(
    loss=loss,
    gamma=gamma,
    results_output=f"{output_dir}/bert_classifier_focal.csv",
    **kwargs,
)

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.175128,0.781350,0.769612
2,0.215100,0.229847,0.801929,0.782294
3,0.130100,0.263660,0.787781,0.763419
4,0.062600,0.437864,0.789068,0.772723
5,0.062600,0.576602,0.786495,0.768369


  0%|          | 0/856 [00:00<?, ?it/s]

...

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.193928,0.736322,0.735194
2,0.212600,0.174399,0.787080,0.781614
3,0.123100,0.330782,0.770600,0.763379
4,0.050800,0.557238,0.767304,0.754534
5,0.050800,0.615424,0.771259,0.759459


  0%|          | 0/973 [00:00<?, ?it/s]

saving the results dataframe to CSV in rumours_output/bert_classifier_focal.csv


In [15]:
bert_classifier

,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,seed,loss,gamma,k_fold
0,0.601491,0.560027,"[0.6950941243582431, 0.4249596557288865]",0.560329,"[0.6915437003405221, 0.42911461162411735]",0.559783,"[0.6986811926605505, 0.42088438998401706]",1,focal,2,True
0,0.620130,0.543593,"[0.7304945781539276, 0.35669191919191917]",0.557802,"[0.6779577810505646, 0.43764523625096824]",0.546435,"[0.7918577981651376, 0.30101225359616407]",12,focal,2,True
0,0.600559,0.548095,"[0.7020714583622966, 0.3941193101498446]",0.550695,"[0.6815114709851552, 0.4198795180722892]",0.547624,"[0.7239105504587156, 0.37133724027703785]",123,focal,2,True


In [16]:
bert_classifier["f1"].mean()

0.5505718409908529

In [17]:
bert_classifier["precision"].mean()

0.5562753863872695

In [18]:
bert_classifier["recall"].mean()

0.5512805708569372

In [19]:
np.stack(bert_classifier["f1_scores"]).mean(axis=0)

array([0.70922005, 0.39192363])

In [20]:
np.stack(bert_classifier["precision_scores"]).mean(axis=0)

array([0.68367098, 0.42887979])

In [21]:
np.stack(bert_classifier["recall_scores"]).mean(axis=0)

array([0.73814985, 0.36441129])

## Using Cross-Entropy loss

In [22]:
loss = "cross_entropy"
gamma = None

In [23]:
bert_classifier_ce = fine_tune_transformer_average_seed(
    loss=loss,
    gamma=gamma,
    results_output=f"{output_dir}/bert_classifier_ce.csv",
    **kwargs,
)

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.534237,0.755627,0.749510
2,0.585600,0.523375,0.804502,0.784382
3,0.405400,0.532079,0.801286,0.782047
4,0.296200,0.760756,0.803215,0.791064
5,0.296200,0.913060,0.798071,0.784738


  0%|          | 0/856 [00:00<?, ?it/s]

...

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.536587,0.748187,0.718452
2,0.603400,0.533451,0.784443,0.769355
3,0.381100,0.763288,0.762690,0.759635
4,0.263900,0.957753,0.780488,0.769722
5,0.263900,1.144476,0.770600,0.757780


  0%|          | 0/973 [00:00<?, ?it/s]

saving the results dataframe to CSV in rumours_output/bert_classifier_ce.csv


In [24]:
bert_classifier_ce

,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,seed,loss,gamma,k_fold
0,0.617521,0.564434,"[0.716496269687759, 0.41237113402061853]",0.568644,"[0.6914666666666667, 0.4458204334365325]",0.563498,"[0.7434059633027523, 0.38359083644112946]",1,cross_entropy,None,True
0,0.623672,0.588426,"[0.7088680605623648, 0.4679841897233201]",0.588033,"[0.7130838410211778, 0.4629822732012513]",0.588899,"[0.7047018348623854, 0.4730953649440597]",12,cross_entropy,None,True
0,0.625909,0.540592,"[0.7385697538100822, 0.34261382246970196]",0.560750,"[0.6767724994031988, 0.4447278911564626]",0.545711,"[0.8127866972477065, 0.27863612147043154]",123,cross_entropy,None,True


In [25]:
bert_classifier_ce["f1"].mean()

0.5644838717123077

In [26]:
bert_classifier_ce["precision"].mean()

0.572475600814215

In [27]:
bert_classifier_ce["recall"].mean()

0.5660361363780776

In [28]:
np.stack(bert_classifier_ce["f1_scores"]).mean(axis=0)

array([0.72131136, 0.40765638])

In [29]:
np.stack(bert_classifier_ce["precision_scores"]).mean(axis=0)

array([0.69377434, 0.45117687])

In [30]:
np.stack(bert_classifier_ce["recall_scores"]).mean(axis=0)

array([0.7536315 , 0.37844077])